In [20]:
import tqdm
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader

In [4]:
# Check if GPU is available, otherwise use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [5]:
# Define the MLP model with one weight matrix and one projection matrix
class SimpleMLP(nn.Module):
    def __init__(self):
        super(SimpleMLP, self).__init__()
        self.weight_matrix = nn.Linear(2, 2)  # First weight matrix (2x2)
        self.projection_matrix = nn.Linear(2, 2)  # Projection layer (2x2)

    def forward(self, x):
        x = self.weight_matrix(x)  # Apply first weight matrix
        x = torch.relu(x)  # Apply ReLU activation
        x = self.projection_matrix(x)  # Apply projection matrix
        return x

# Initialize the model, loss function, and optimizer
model = SimpleMLP().to(device)
criterion = nn.MSELoss()  # Using Mean Squared Error for regression
optimizer = optim.SGD(model.parameters(), lr=0.01)  # Stochastic Gradient Descent

In [ ]:
# Example training data (2D input -> 2D output)
inputs = torch.tensor([[1.0, 2.0], [3.0, 4.0], [-1.0, -2.0], [-3.0, -4.0]], device=device)
targets = torch.tensor([[2.0, 4.0], [6.0, 8.0], [-2.0, -4.0], [-6.0, -8.0]], device=device)

In [14]:
class CustomCSVSequenceDataset(Dataset):
    def __init__(self, file_paths):
        """
        Args:
            file_paths (list of str): List of paths to CSV files to load.
        """
        self.data = []

        # Load data from all CSV files
        for path in file_paths:
            df = pd.read_csv(path, usecols=[1, 2])  # Load only columns 1 and 2
            # Convert to list of tuples [(col1_val, col2_val), ...]
            self.data.extend(df.values.tolist())

    def __len__(self):
        """Return the total number of samples in the dataset."""
        return len(self.data) - 1  # One less because we look at (i, i+1)

    def __getitem__(self, index):
        """
        Args:
            index (int): Index of the sample to retrieve.

        Returns:
            tuple: (input_tensor, label_tensor)
        """
        # Input is row i, Label is row i + 1
        input_data = torch.tensor(self.data[index], dtype=torch.float32)
        label_data = torch.tensor(self.data[index + 1], dtype=torch.float32)

        return input_data, label_data

In [26]:
# Load the training data
file_paths = ["../data/predator_prey_solution_14_11.csv", 
              "../data/predator_prey_solution_17_57.csv",
              "../data/predator_prey_solution_24_15.csv"]  # Replace with your actual file paths
dataset = CustomCSVSequenceDataset(file_paths)

# Create a Training DataLoader to iterate over the dataset
train_loader = DataLoader(dataset, batch_size=128, shuffle=True)


# Load the validation data
file_paths = ["../data/predator_prey_solution_25_72.csv", 
              "../data/predator_prey_solution_40_40.csv",
              "../data/predator_prey_solution_44_52.csv"]  # Replace with your actual file paths
val_dataset = CustomCSVSequenceDataset(file_paths)

# Create a Training DataLoader to iterate over the dataset
val_loader = DataLoader(dataset, batch_size=128, shuffle=True)

# # Iterate through the dataset and print some samples
# for inputs, labels in dataloader:
#     print(f"Inputs: {inputs}\nLabels: {labels}")
#     break  # Print only the first batch for inspection

In [24]:
# # Load training data from data files
# reader = csv.reader()
# header = next(reader)

# inputs = list()
# targets = list()
# for i, row in enumerate(reader):
#     inputs.append(row[1:])
#     if i != 0:
#         targets.append(inputs[-1])

In [28]:
# Training loop
## FIX THIS
num_epochs = 1000  # Number of epochs to train the model
for epoch in tqdm.tqdm(range(num_epochs)):

    
    for batch in train_loader:
        inputs, targets = batch
        # Forward pass
        outputs = model(inputs.to(device))
        loss = criterion(outputs, targets.to(device))

        # Backward pass and optimization
        optimizer.zero_grad()  # Zero out gradients
        loss.backward()  # Backpropagate the error
        optimizer.step()  # Update model parameters

    # Print the loss every 10 epochs
    if (epoch + 1) % 10 == 0:
        tqdm.tqdm.write(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Test the model with new inputs
test_input = torch.tensor([[5.0, 6.0]], device=device)
predicted_output = model(test_input)
print(f"Predicted output for {test_input.cpu().numpy()}: {predicted_output.cpu().detach().numpy()}")

  2%|▏         | 15/1000 [00:00<00:35, 27.99it/s]

Epoch [10/1000], Loss: 207.3623


  2%|▏         | 24/1000 [00:00<00:34, 27.90it/s]

Epoch [20/1000], Loss: 165.2606


  3%|▎         | 33/1000 [00:01<00:34, 27.77it/s]

...

 63%|██████▎   | 633/1000 [00:22<00:13, 27.73it/s]

Epoch [630/1000], Loss: 177.9777


 64%|██████▍   | 642/1000 [00:23<00:12, 27.85it/s]

Epoch [640/1000], Loss: 118.6276


KeyboardInterrupt: 